<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?" data-toc-modified-id="Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Преобразование-данных" data-toc-modified-id="Преобразование-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Преобразование данных</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Обучение модели</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [29]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_spd_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv('/datasets/insurance.csv')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


Нужно провести предоработку. Проверить пропуски, дубликаты.

In [31]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [32]:
df.duplicated().sum()

153

Пропусков нет, а вот дубликатов целых 153. Посмотрим на них поближе.

In [33]:
df[df.duplicated()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


In [34]:
print(df[df['Зарплата'] == 48100.0])

      Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
69      1     39.0   48100.0            1                  0
281     1     39.0   48100.0            1                  0
434     1     32.0   48100.0            2                  0
717     1     27.0   48100.0            3                  0
906     0     27.0   48100.0            0                  0
1081    1     42.0   48100.0            0                  1
1263    0     44.0   48100.0            0                  1
1279    0     44.0   48100.0            3                  1
1706    0     29.0   48100.0            3                  0
1763    1     29.0   48100.0            2                  0
2172    0     38.0   48100.0            1                  0
2419    1     34.0   48100.0            0                  0
3044    1     20.0   48100.0            2                  0
3392    0     25.0   48100.0            0                  0
4043    1     29.0   48100.0            2                  0
4215    1     24.0   481

Из полученных данных могу сделать вывод, что дубликаты удалять не стоит. В нашем наборе данных не так много столбцов, чтобы предположить невозможность случайного совпадения данных. Данные действительно могут совпадать у некоторых клиентов. Поскольку у нас нет id клинета, то невозможно в этом убедиться наверняка. Дубликаты сокрее всего не являются дубликатами, поэтмоу не будем их удалять. 

Сразу поделим наш сет на признаки и целевой признак:

In [35]:
features = df.drop(columns=['Страховые выплаты'])
target = df['Страховые выплаты']

## Умножение матриц

### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** 


При подстановке в формулы вместо X - XP, получаем:


Предсказания:

$$
a = XPw
$$

Формула обучения:

$$
w = ((XP)^T XP)^{-1} XP^T y
$$

В результате чего формула предсказания будет иметь вид:

$$
a = XP((XP)^T XP)^{-1} XP^T y
$$

При раскрытии скобок меняется порядок умножения для транспонируемых матриц:

$$
a = XP(P^T X^T XP)^{-1} P^T X^T y
$$


Также меняется порядок умножения при раскрытии скобок для инверсии:

$$
a = XP(P)^{-1} (P^T X^T X)^{-1} P^T X^T y
$$


$$
a = XP(P)^{-1} (X^T X)^{-1} P^{T^{-1}} P^T X^T y
$$


Произведение матрицы на обратную дает еденичную матрицу:

$$
a = XE(X^T X)^{-1} EX^T y
$$

И умножение на единичную матрицу оставляет исходную матрицу, получаем:

$$
a = X(X^TX)^{-1} X^T y
$$

Что равно формуле предсказаний для исходного набора признаков:

$$
a = Xw
$$

## Алгоритм преобразования

**Алгоритм**

Для защиты данных клиентов мы долждны создать алгоритм шифровки и дешифровки данных таким образом, чтобы преобразования не мешали алгоритмам машинного обучения. Для этого нам нужно создать объект, который принимает на вход признаки и возвращает обработанную матрицу, и в то же время возвращать оригинальные данные при подаче обработанных.

При подаче на вход необработанных признаков, будет случайным образом генерироваться обратимая матрица нужного размера, и обратная ей, для последующей расшифровки. Матрицы должны быть спрятаны внутри объекта

**Обоснование**

Из ответа на предыдущий вопрос, мы знаем, что при обучении модели линейной регрессии, и последующем предсказании, влияние матрицы, на которую умножаются признаки, сходит на нет.


Но для этого признаки, для которых предсказываются значения, должны быть умножены на ту же самую матрицу, на которую умножались признаки для обучения модели.

## Проверка алгоритма

### Преобразование данных

Создадим матрицу признаков:

In [36]:
features_matrix = features.values
features_matrix

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Создадим случайную матрицу для дальнейших преобразований данных:

In [37]:
random_matrix = make_spd_matrix(n_dim=4, random_state=123)
random_matrix

array([[ 0.47243659,  0.36201499,  0.23624413, -0.20756552],
       [ 0.36201499,  3.26723888,  1.70966795, -0.39838002],
       [ 0.23624413,  1.70966795,  1.32704651, -0.17893753],
       [-0.20756552, -0.39838002, -0.17893753,  0.35460046]])

Проверим случайную матрицу на обратимость:

In [38]:
np.linalg.inv(random_matrix)

array([[ 2.97675307,  0.09601407, -0.43363796,  1.63149048],
       [ 0.09601407,  1.02415391, -1.2595126 ,  0.57122764],
       [-0.43363796, -1.2595126 ,  2.39108163, -0.46226306],
       [ 1.63149048,  0.57122764, -0.46226306,  4.18355526]])

Ошибки не возникло, следовательно матрица обратима. 

Умножим матрицы, для преобразования данных. 

In [39]:
code_matrix = features_matrix @ random_matrix
code_matrix

array([[11732.81637505, 84933.45064883, 65891.66076973, -8891.48812605],
       [ 8993.72209566, 65117.27663171, 50506.23330695, -6817.59709104],
       [ 4971.62518218, 35997.77683511, 27917.55715724, -3769.24118958],
       ...,
       [ 8015.50120409, 58022.29145405, 45020.71229678, -6073.24072949],
       [ 7732.99714812, 55977.18802921, 43431.73312289, -5859.16541635],
       [ 9601.91300269, 69503.96501156, 53926.01646352, -7275.87139908]])

Данные преобразовали, можно приступать к обучению модели.

### Обучение модели

Разделим выборки на тренировочную и валидационную:

In [40]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, train_size=0.75, test_size=0.25, random_state=123)

Обучим модель линейной регрессии и посчитаем метрику R2

In [41]:
model_lr = LinearRegression()
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
r2 = r2_score(target_valid, predictions_lr)

print('Метрика R2 до преобразования данных:', r2)

Метрика R2 до преобразования данных: 0.43018469990933306


Проверим метрику на преобразованных данных. Сначала распределим данные на обучающую и валидационную:

In [42]:
features_train_code, features_valid_code, target_train_code, target_valid_code = train_test_split(
    code_matrix, target, train_size=0.75, test_size=0.25, random_state=123)

И обучим модель линейной регрессии:

In [43]:
model_code = LinearRegression()
model_code.fit(features_train_code, target_train_code)
predictions_code = model_code.predict(features_valid_code)

r2_code = r2_score(target_valid_code, predictions_code)
print('Метрика R2 после преобразования данных:', r2_code)

Метрика R2 после преобразования данных: 0.4301846999101159


Сравним метрики:

In [44]:
print(r2)
print(r2_code)

0.43018469990933306
0.4301846999101159


Метрики отличаются на 11 знаке после запятой, значит, они практически идентичны, что свидетельствует о верном перобразовании данных, и их ревнозначности.

## Вывод

Во время проекта выполнили следующие шаги:

- загрузили и изучили данные
- математически доказали, что качество линейной регрессии не изменится, если умножать признаки на обратимую матрицу
- предложили алгоритм преобразования данных и протестировали его работу. Удалось доказать, что качество модели линейной регрессии после преобразования данных не изменилось.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования